In [ ]:
import pandas as pd

In [ ]:
df = pd.read_pickle('./remarks_data.pickle')

In [ ]:
df.head(2)

In [ ]:
# groupby 
df = df.groupby('Occurrence Remarks').mean().reset_index()

In [ ]:
df = df[df['Expert Field Note Rating'] > 0]

In [ ]:
df

In [ ]:
#df['Expert Remark Rating'] /= 5 # normalize output to 0-1; brackets of 0.2 each

#### Build Train & Test Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df['Expert Field Note Rating'].values, 
                                                  test_size=0.10, 
                                                  random_state=42, 
                                                  )

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
X_train = df[df.data_type=='train']['Occurrence Remarks'].values
X_val = df[df.data_type=='val']['Occurrence Remarks'].values
y_train = df[df.data_type=='train']['Expert Field Note Rating'].values
y_val = df[df.data_type=='val']['Expert Field Note Rating'].values

In [ ]:
# try to undersample
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_train, y_train = rus.fit_resample(X_train.reshape(-1,1), y_train.reshape(-1,1))
X_train = X_train.reshape(-1,1)
y_train = y_train.reshape(-1,1)
X_train.shape, y_train.shape

In [ ]:
import numpy as np
y_train = y_train.astype(np.float64)
y_train -= 2.5
y_train /= 2.5
y_val = y_val.astype(np.float64)
y_val -= 2.5
y_val /= 2.5

In [ ]:
import random
import numpy as np 
import torch
import tensorflow as tf

seed_val = 17
tf.random.set_seed(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)

#### Try AutoKeras for Regression
https://autokeras.com/tutorial/text_regression/

In [ ]:
# !pip install autokeras --user

In [ ]:
import autokeras as ak
# Initialize the text regressor.
reg = ak.TextRegressor(overwrite=True, max_trials=1)  # It tries 10 different models.

Fit with our data

In [ ]:
# Feed the text regressor with training data.
reg.fit(X_train, y_train, epochs=5)
# Predict with the best model.
predicted_y = reg.predict(X_val)
# Evaluate the best model with testing data.
print(reg.evaluate(X_val, y_val))

In [ ]:
predicted_y

In [ ]:
pd.DataFrame([X_val, predicted_y.reshape(-1,1)]).T

Seems all output results are similar.. Model not learning to differentiate, instead just minimising the loss. <br>
Perhaps not enough training data.print(type(model))  # <class 'tensorflow.python.keras.engine.training.Model'>

try:
    model.save("model_autokeras", save_format="tf")
except Exception:
    model.save("model_autokeras.h5")

In [ ]:
# Export as a Keras Model.
model = reg.export_model()

print(type(model))  # <class 'tensorflow.python.keras.engine.training.Model'>

try:
    model.save("model_autokeras", save_format="tf")
except Exception:
    model.save("model_autokeras.h5")

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

In [ ]:
predicted_y = loaded_model.predict(tf.expand_dims(np.array([''''Pruning' a young plum tree in a suburban backyard at dinner time.
Six birds perched quietly in or on the trees. Quiet isn't usually how I'd describe them but they were very busy.
Less than 10m from an artificial water source. Less than 2km from a natural water source. Not near a major road or rocky outcrop.''']), -1))
print(predicted_y)